In [ ]:
#=
using Pkg; 
packages = ["Knet", "AutoGrad", "Random", "Test", "MLDatasets", "CUDA", "Plots", "GR","Statistics",
            "IterTools", "StatsBase", "DSP", "Images", "DelimitedFiles", "MultivariateStats", "PyPlot", "PyCall"];
Pkg.add(packages);
=#

In [1]:
include("../src/modules/TUM69.jl")
include("../src/modules/Preprocess.jl")
include("../src/modules/Network.jl")
include("../src/modules/Utils.jl")

## Third party packages
using Knet: KnetArray, adam, relu, minibatch
using AutoGrad, Knet, CUDA


## Handwritten modules
using .TUM69: load_accel_data   # Data reading
using .Preprocess: process_accel_signal # Preprocessing on the data
using .Network: GCN, train_summarize!, accuracy4, nll4, GenericMLP # Construction of custom network
using .Utils: notify

In [2]:
AutoGrad.set_gc_function(AutoGrad.default_gc)

default_gc (generic function with 1 method)

In [3]:
# Trick from Deniz Hoca to deal with this issue: https://github.com/denizyuret/Knet.jl/issues/524
#=
using Knet
function Knet.KnetArray(x::CuArray{T,N}) where {T,N}
    p = Base.bitcast(Knet.Cptr, pointer(x))
    k = Knet.KnetPtr(p, sizeof(x), Int(CUDA.device().handle), x)
    KnetArray{T,N}(k, size(x))
end
=#

In [4]:
# Array type setting for GPU usage
a_type() = (CUDA.functional() ? KnetArray{Float32} : Array{Float32})

a_type (generic function with 1 method)

In [5]:
# CUDA information
GC.gc(true)
CUDA.device()

CuDevice(0): Tesla V100-PCIE-32GB

In [6]:
path = CUDA.functional() ? "/userfiles/vaydingul20/data/trial" : "./../data/trial" # path of the main data
DATA_PATH = isdir(path) && path

"/userfiles/vaydingul20/data/trial"

In [7]:
X_train, y_train,
X_test, y_test, 
material_dict = @time load_accel_data(DATA_PATH; mode = "baseline");  # Data loading routine

Train accel data is being loaded!
Test accel data is being loaded!
  0.883581 seconds (4.89 M allocations: 167.417 MiB, 4.73% gc time)


In [8]:
println("X_train = ", summary(X_train))
println("y_train = ", summary(y_train))
println("X_test  = ", summary(X_test))
println("y_test  = ", summary(y_test))
println("material_dict = ", summary(material_dict))

X_train = 3-element Array{Array{Float32,1},1}
y_train = 3-element Array{Int8,1}
X_test  = 3-element Array{Array{Float32,1},1}
y_test  = 3-element Array{Int8,1}
material_dict = Dict{String,Int8} with 3 entries


In [9]:
#Preprocessing on the acceleration data
@time X_train_modified, y_train_modified = process_accel_signal(X_train, y_train);
@time X_test_modified, y_test_modified = process_accel_signal(X_test, y_test);

  3.484892 seconds (6.87 M allocations: 364.186 MiB, 4.55% gc time)
  0.037690 seconds (388 allocations: 30.432 MiB)


In [10]:
println("X_train = ", summary(X_train_modified))
println("y_train = ", summary(y_train_modified))
println("X_test  = ", summary(X_test_modified))
println("y_test  = ", summary(y_test_modified))
println("material_dict = ", summary(material_dict))

X_train = 50×300×1×24 Array{Float32,4}
y_train = 24-element Array{Int8,1}
X_test  = 50×300×1×24 Array{Float32,4}
y_test  = 24-element Array{Int8,1}
material_dict = Dict{String,Int8} with 3 entries


In [11]:
# Some constants that will be used in the network model
MINIBATCH_SIZE = 10
INPUT_SIZE = size(X_test_modified)[1:3]
OUTPUT_SIZE = size(collect(keys(material_dict)))[1];

In [12]:
# Minibatching
dtrn = minibatch(X_train_modified, y_train_modified, MINIBATCH_SIZE; xtype = a_type(), shuffle = true)
dtst = minibatch(X_test_modified, y_test_modified, MINIBATCH_SIZE; xtype = a_type(), shuffle = true);

In [13]:
#=
model = GCN(INPUT_SIZE, OUTPUT_SIZE, 
       [(25, 150, 1 , relu, 0.0, (1, 1), (1, 1), (2, 2), false)]; 
    hidden=[10], optimizer_type = sgd, lr = 0.15, loss_fnc=nll, accuracy_fnc = accuracy, atype=a_type())
=#

In [14]:

model = GCN(INPUT_SIZE, OUTPUT_SIZE, 
       [(3, 3, 50 , relu, 0.0, (1, 1), (1, 1), (2, 2), true),
        (3, 3, 100, relu, 0.0, (1, 1), (1, 1), (2, 2), false),
        (3, 3, 150, relu, 0.0, (1, 1), (1, 1), (2, 2), false),
        (3, 3, 200, relu, 0.0, (1, 1), (1, 1), (2, 2), false),
        (4, 12,400, relu, 0.5, (1, 0), (1, 1), (1, 1), false),
        (1, 1, 250, relu, 0.5, (0, 0), (1, 1), (1, 1), false),
        (1, 1, OUTPUT_SIZE , relu, 0.5, (0, 0), (1, 1), (1, 1), false)
        ]; 
    hidden=[], optimizer_type = adam, lr = 1e-4, loss_fnc=nll4, accuracy_fnc = accuracy4, atype=a_type())

#=
model = GCN(INPUT_SIZE, OUTPUT_SIZE, 
       [(3, 3, 50 , relu, 0.0, (1, 1), (1, 1), (2, 2), true)
        ]; 
    hidden=[], optimizer_type = adam, lr = 1e-4, loss_fnc=nll4, accuracy_fnc = accuracy4, atype=a_type())
=#

GCN((Main.Network.Conv(P(KnetArray{Float32,4}(3,3,1,50)), P(KnetArray{Float32,4}(1,1,50,1)), Knet.Ops20.relu, 0, (1, 1), (1, 1), (2, 2), true, KnetArray{Float32,N} where N), Main.Network.Conv(P(KnetArray{Float32,4}(3,3,50,100)), P(KnetArray{Float32,4}(1,1,100,1)), Knet.Ops20.relu, 0, (1, 1), (1, 1), (2, 2), false, KnetArray{Float32,N} where N), Main.Network.Conv(P(KnetArray{Float32,4}(3,3,100,150)), P(KnetArray{Float32,4}(1,1,150,1)), Knet.Ops20.relu, 0, (1, 1), (1, 1), (2, 2), false, KnetArray{Float32,N} where N), Main.Network.Conv(P(KnetArray{Float32,4}(3,3,150,200)), P(KnetArray{Float32,4}(1,1,200,1)), Knet.Ops20.relu, 0, (1, 1), (1, 1), (2, 2), false, KnetArray{Float32,N} where N), Main.Network.Conv(P(KnetArray{Float32,4}(4,12,200,400)), P(KnetArray{Float32,4}(1,1,400,1)), Knet.Ops20.relu, 0, (1, 0), (1, 1), (1, 1), false, KnetArray{Float32,N} where N), Main.Network.Conv(P(KnetArray{Float32,4}(1,1,400,250)), P(KnetArray{Float32,4}(1,1,250,1)), Knet.Ops20.relu, 0, (0, 0), (1, 1), (1

In [18]:
model.layers[1](collect(dtrn)[1][1])

25×150×50×10 KnetArray{Float32,4}:
[:, :, 1, 1] =
 0.00929634  0.0         0.025561    …  0.0          0.0        0.065289
 0.02804     0.0243259   0.00298508     0.0207448    0.0359301  0.0614998
 0.0         0.033067    0.059477       0.0182257    0.0        0.0783443
 0.0         0.0525847   0.00706627     0.000450538  0.0290402  0.026758
 0.0         0.0         0.035943       0.0          0.010466   0.021475
 0.0         0.00651828  0.0218636   …  0.0          0.0251495  0.0349209
 0.0         0.0154886   0.0193628      0.0          0.0174371  0.0243134
 0.0         0.00531616  0.0243444      0.0          0.0175766  0.0220975
 0.0         0.00716122  0.0133045      0.0          0.0169098  0.0195291
 0.0         0.00668463  0.0157864      0.0          0.0131735  0.0198256
 0.0         0.00686225  0.0135191   …  0.0          0.014144   0.0195979
 0.0         0.0063082   0.0145264      0.0          0.0151957  0.0220079
 0.0         0.00611751  0.0145572      0.0          0.0155835  0

In [16]:
notify("OK!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

{}

100     5  100     2  100     3      1      2  0:00:02  0:00:01  0:00:01     4


Process(`curl https://notify.run/fnx04zT7QmOlLLa6 -d OK!`, ProcessExited(0))

In [19]:
# Training routine
# Currently, the model is not working due to the issue mentioned in: https://github.com/denizyuret/Knet.jl/issues/624#
# As soon as it is solved, I hope the model will be accurately working.
res = train_summarize!(model, dtrn, dtst; 
                       train_type = "epoch", progress_bar = true ,fig = true, info = true, 
                       epoch = 1, conv_epoch = 50, max_conv_cycle = 20)


Stacktrace:
 [1] LR_norm(::AutoGrad.Result{KnetArray{Float32,4}}; k::Int64, n::Int64, alpha::Float64, beta::Float64, el_type::Type{T} where T) at /scratch/users/vaydingul20/workfolder/COMP541_Project/src/modules/Network.jl:467
 [2] LR_norm(::AutoGrad.Result{KnetArray{Float32,4}}) at /scratch/users/vaydingul20/workfolder/COMP541_Project/src/modules/Network.jl:461
 [3] (::Main.Network.Conv)(::KnetArray{Float32,4}) at /scratch/users/vaydingul20/workfolder/COMP541_Project/src/modules/Network.jl:232
 [4] (::GCN)(::KnetArray{Float32,4}) at /scratch/users/vaydingul20/workfolder/COMP541_Project/src/modules/Network.jl:305
 [5] (::GCN)(::KnetArray{Float32,4}, ::Array{Int8,1}) at /scratch/users/vaydingul20/workfolder/COMP541_Project/src/modules/Network.jl:316
 [6] (::Knet.Train20.var"#27#28"{Knet.Train20.Minimize{IterTools.NCycle{Knet.Train20.Data{Tuple{KnetArray{Float32,N} where N,Array{Int8,N} where N}}}},Tuple{KnetArray{Float32,4},Array{Int8,1}}})() at /kuacc/users/vaydingul20/.julia/packages

LoadError: MethodError: Cannot `convert` an object of type Array{Float32,4} to an object of type AutoGrad.Result{KnetArray{Float32,4}}
Closest candidates are:
  convert(::Type{T}, !Matched::Intervals.Interval{T,L,R} where R<:Intervals.Bound where L<:Intervals.Bound) where T at /kuacc/users/vaydingul20/.julia/packages/Intervals/ua9cq/src/interval.jl:253
  convert(::Type{T}, !Matched::Intervals.AnchoredInterval{P,T,L,R} where R<:Intervals.Bounded where L<:Intervals.Bounded) where {P, T} at /kuacc/users/vaydingul20/.julia/packages/Intervals/ua9cq/src/anchoredinterval.jl:181
  convert(::Type{T}, !Matched::T) where T at essentials.jl:171
  ...

In [22]:
k = randn(4,4,4,4)
convert(a_type(), k)

4×4×4×4 KnetArray{Float32,4}:
[:, :, 1, 1] =
  0.402288     1.4645     1.38384   -1.39481
 -0.00514829  -0.324605  -2.11427   -2.22092
  1.28046      0.316134  -0.679059  -2.2797
 -1.07817     -1.98771    1.25159   -1.4233

[:, :, 2, 1] =
  0.659342  -2.16028   0.0734077  0.991875
 -1.08108    0.311909  0.764703   0.0737313
  1.39673    2.33551   0.562021   1.94637
 -0.726634  -0.944274  0.130176   1.14163

[:, :, 3, 1] =
 -0.813941   0.941153  -1.10348    1.50984
 -1.41253   -1.83948   -0.753016   2.84139
  0.142604   0.549539   0.771396   0.532946
 -0.640998   1.24086    0.741711  -0.524934

[:, :, 4, 1] =
  0.449322   0.159099   0.918386  1.09786
 -1.08158   -1.18002   -0.681726  0.394099
 -1.34624   -0.683723   1.9625    0.521123
  1.15276    0.778985   2.40675   0.573061

[:, :, 1, 2] =
  0.150849  -0.205895    0.242364   -0.275909
 -0.386792  -0.732285   -1.67593     0.680515
 -0.222195   0.799617   -0.0731401   0.00857272
 -0.998263   0.0511593   0.0254469  -0.425287

[:, :, 2, 

In [ ]:
notify("OK!")